Tiles []
Bounding box for intersections []
Loop through tiles and check if bounding boxes collide
List out to a file all of the bounding boxes for intersections that collide with tiles.

In [ ]:
Understanding my code more and validation check everything 
fourth attempt v

In [1]:
import geopandas as gpd
import json
import rasterio
from rasterio.warp import transform_bounds
from osgeo import ogr, osr
import os
import csv

In [2]:
# Directory containing the .tiff files
DATA_DIR = "./data/"
tif_folder = DATA_DIR + "B5/"

# Load the GeoJSON file
with open('./Transmission_Line_1604355110918241857.geojson') as f:
    data = json.load(f)

# Create a GeoDataFrame from the GeoJSON data
gdf = gpd.GeoDataFrame.from_features(data['features'], crs='EPSG:4269')

# Obtain the bounding boxes
bounding_boxes = gdf.bounds

# Target CRS (EPSG code of GeoJSON data)
target_crs = 'EPSG:32614'

# List to store bounding boxes of tiles as OGR Geometry objects
tiles = []

In [3]:
print(f"Checking '{bounding_boxes}'...")
print("")
print(f"Checking '{tiles}'...")
print("")

Checking '              minx          miny          maxx          maxy
0    -1.360804e+07  4.545689e+06 -1.360485e+07  4.547327e+06
1    -1.361356e+07  4.547254e+06 -1.360804e+07  4.548298e+06
2    -1.361343e+07  4.548276e+06 -1.361324e+07  4.549615e+06
3    -1.361313e+07  4.549619e+06 -1.361173e+07  4.550724e+06
4    -1.300083e+07  3.963888e+06 -1.298179e+07  3.989232e+06
...            ...           ...           ...           ...
6834 -1.340772e+07  4.532550e+06 -1.340720e+07  4.532861e+06
6835 -1.340771e+07  4.532568e+06 -1.340716e+07  4.532891e+06
6836 -1.346811e+07  4.516440e+06 -1.345261e+07  4.522351e+06
6837 -1.346811e+07  4.516440e+06 -1.345261e+07  4.522351e+06
6838 -1.358993e+07  4.611654e+06 -1.358558e+07  4.614082e+06

[6839 rows x 4 columns]'...

Checking '[]'...



In [4]:
# Function to create OGR Geometry object from bounding box
def create_ogr_geometry(minx, miny, maxx, maxy):
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(minx, miny)
    ring.AddPoint(maxx, miny)
    ring.AddPoint(maxx, maxy)
    ring.AddPoint(minx, maxy)
    ring.AddPoint(minx, miny)
    polygon = ogr.Geometry(ogr.wkbPolygon)
    polygon.AddGeometry(ring)
    return polygon

In [5]:
# Function to reproject bounding box coordinates
#plotlly and matplotlib
#plot everything before and after
def reproject_bounds(src_bounds, src_crs, dst_crs):
    minx, miny, maxx, maxy = src_bounds
    dst_minx, dst_miny, dst_maxx, dst_maxy = transform_bounds(
        src_crs, dst_crs, minx, miny, maxx, maxy
    )
    return dst_minx, dst_miny, dst_maxx, dst_maxy

In [6]:
# Loop through all .tiff files in the folder
for filename in os.listdir(tif_folder):
    if filename.endswith('.TIF'):
        filepath = os.path.join(tif_folder, filename)
        print(f"Opening '{filepath}'...")
        with rasterio.open(filepath) as src:
            src_crs = src.crs
            minx, miny, maxx, maxy = reproject_bounds(src.bounds, src_crs, target_crs)
            raster_box = create_ogr_geometry(minx, miny, maxx, maxy)
            tiles.append(raster_box)
            print(raster_box, "\n")

Opening './data/B5/LC08_L2SP_039035_20240513_20240521_02_T1_SR_B5.TIF'...
POLYGON Z ((-1068768.32130903 3968394.24856558 0,-795124.606744511 3968394.24856558 0,-795124.606744511 4245453.73485258 0,-1068768.32130903 4245453.73485258 0,-1068768.32130903 3968394.24856558 0)) 

Opening './data/B5/LC08_L2SP_039036_20240513_20240521_02_T1_SR_B5.TIF'...
POLYGON Z ((-1133116.57901433 3813735.74931211 0,-859628.791997485 3813735.74931211 0,-859628.791997485 4090679.45046075 0,-1133116.57901433 4090679.45046075 0,-1133116.57901433 3813735.74931211 0)) 

Opening './data/B5/LC08_L2SP_039037_20240513_20240521_02_T1_SR_B5.TIF'...
POLYGON Z ((-1197196.53677373 3658716.97697197 0,-923885.441892183 3658716.97697197 0,-923885.441892183 3935524.98132793 0,-1197196.53677373 3935524.98132793 0,-1197196.53677373 3658716.97697197 0)) 

Opening './data/B5/LC08_L2SP_041034_20240511_20240520_02_T1_SR_B5.TIF'...
POLYGON Z ((-1285656.00755186 4169331.05518694 0,-1003108.96753288 4169331.05518694 0,-1003108.967532

In [7]:
#plot crs and bounding box on top
print(f"Checking '{src}'...")
print("")
print(f"Checking '{raster_box}'...")
print("")
print(f"Checking '{tiles}'...")
print("")

Checking '<closed DatasetReader name='./data/B5/LC09_L2SP_046031_20240522_20240524_02_T1_SR_B5.TIF' mode='r'>'...

Checking 'POLYGON Z ((-1730104.15620155 4778721.09919903 0,-1421765.84606161 4778721.09919903 0,-1421765.84606161 5089161.54170665 0,-1730104.15620155 5089161.54170665 0,-1730104.15620155 4778721.09919903 0))'...

Checking '[<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f76fc0750b0> >, <osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f74b97b3d80> >, <osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f76fc0533c0> >, <osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f74af01c2a0> >, <osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f74af01cea0> >, <osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f74af01cff0> >, <osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f74af01ced0

In [8]:
# Function to check for intersection between bounding boxes
def check_intersection(bbox, tile_geom):
    minx, miny, maxx, maxy = bbox.minx, bbox.miny, bbox.maxx, bbox.maxy
    bbox_geom = create_ogr_geometry(minx, miny, maxx, maxy)
    return bbox_geom.Intersects(tile_geom)

# List to store intersecting bounding boxes
intersections = []

In [9]:
for bbox in bounding_boxes.itertuples():
    for tile_bbox in tiles:
        if check_intersection(bbox, tile_bbox):
            intersections.append(bbox)


In [10]:
print(f"Checking '{bbox}'...")
print("")
print(f"Checking '{intersections}'...")
print("")

Checking 'Pandas(Index=6838, minx=-13589927.8525, miny=4611654.1394, maxx=-13585580.0669, maxy=4614081.9809)'...

Checking '[]'...



In [11]:
# Write the intersecting bounding boxes to a file
output_file = 'intersecting_bounding_boxes.csv'
with open(output_file, 'w') as f:
    #instead of a for loop just write out intersections
    for bbox in intersections:
        f.write(f'{bbox},\n')

I am not sure which one of these works better since I can not test the yet since there are no intersections ^v

In [ ]:
# Write the intersecting bounding boxes to a CSV file
output_file = 'intersecting_bounding_boxes.csv'
with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['minx', 'miny', 'maxx', 'maxy'])
    for bbox in intersections:
        csvwriter.writerow([bbox.minx, bbox.miny, bbox.maxx, bbox.maxy])